### Creating a Delta Table with SQL DDL

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

from delta import *

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]
warehouse_dir = f"{work_dir}/spark-warehouse"

# Create spark session with hive enabled
builder = (
    SparkSession
    .builder
    .appName("pyspark-notebook")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .enableHiveSupport()
)

In [2]:
# 델타 레이크 생성시에 반드시 `configure_spark_with_delta_pip` 구성을 통해 실행되어야 정상적인 델타 의존성이 로딩됩니다
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

In [4]:
print(work_dir)
!ls /home/jovyan/work/data/rateCard.delta

/home/jovyan/work
ls: cannot access '/home/jovyan/work/data/rateCard': No such file or directory


In [5]:
createTableWithPath=f"""
CREATE TABLE IF NOT EXISTS delta.`{work_dir}/data/rateCard.delta` (
    rateCodeId INT,
    rateCodeDesc STRING
)
USING DELTA
"""
spark.sql(createTableWithPath)

!ls {work_dir}/data/rateCard.delta/_delta_log
!cat {work_dir}/data/rateCard.delta/_delta_log/00000000000000000000.json

00000000000000000000.json  00000000000000000001.json
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"06b115e0-2fac-41de-9686-6a0d4700de9e","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"rateCodeId\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"rateCodeDesc\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1723625005545}}
{"commitInfo":{"timestamp":1723625005712,"operation":"CREATE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{},"engineInfo":"Apache-Spark/3.2.1 Delta-Lake/2.0.0","txnId":"16721413-dd29-4e86-aa5e-510d9628a818"}}


In [6]:
spark.sql("show databases")

namespace
default
taxidb


In [7]:
spark.sql("show tables")

namespace,tableName,isTemporary
default,family,false
default,users,false


In [8]:
createDatabase=f"""
CREATE DATABASE IF NOT EXISTS taxidb;
"""
spark.sql(createDatabase)

createTableWithTable=f"""
CREATE TABLE IF NOT EXISTS taxidb.rateCard (
    rateCodeId INT,
    rateCodeDesc STRING
) USING DELTA
LOCATION '{work_dir}/data/rateCard.delta'
"""
spark.sql(createTableWithTable)

""


In [9]:
spark.sql("use taxidb")
spark.sql("show tables")

namespace,tableName,isTemporary
taxidb,greentaxis,false
taxidb,ratecard,false


In [10]:
spark.sql("describe extended ratecard").show(truncate=False)
spark.sql("select * from ratecard").show(truncate=False)

+----------------------------+-------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                      |comment|
+----------------------------+-------------------------------------------------------------------------------+-------+
|rateCodeId                  |int                                                                            |       |
|rateCodeDesc                |string                                                                         |       |
|                            |                                                                               |       |
|# Partitioning              |                                                                               |       |
|Not partitioned             |                                                                               |       |
|                            |                  

In [12]:
csvRateCard = spark.read.format("csv").option("inferSchema", True).option("header", True).load(f"{work_dir}/data/rateCard.csv")
csvRateCard.printSchema()
csvRateCard.show(truncate=False)

root
 |-- RateCodeID: integer (nullable = true)
 |-- RateCodeDesc: string (nullable = true)

+----------+---------------------+
|RateCodeID|RateCodeDesc         |
+----------+---------------------+
|1         |Standard Rate        |
|2         |JFK                  |
|3         |Newark               |
|4         |Nassau or Westchester|
|5         |Negotiated fare      |
|6         |Group ride           |
+----------+---------------------+



In [13]:
csvRateCard.write.format("delta").mode("overwrite").saveAsTable("taxidb.rateCard")

In [14]:
!ls {work_dir}/data/rateCard.delta/_delta_log
!cat {work_dir}/data/rateCard.delta/_delta_log/00000000000000000001.json

00000000000000000000.json  00000000000000000001.json
{"add":{"path":"part-00000-879dbb89-ee5a-47d4-ab0d-0e6445663607-c000.snappy.parquet","partitionValues":{},"size":858,"modificationTime":1723625669220,"dataChange":true,"stats":"{\"numRecords\":6,\"minValues\":{\"rateCodeId\":1,\"rateCodeDesc\":\"Group ride\"},\"maxValues\":{\"rateCodeId\":6,\"rateCodeDesc\":\"Standard Rate\"},\"nullCount\":{\"rateCodeId\":0,\"rateCodeDesc\":0}}"}}
{"commitInfo":{"timestamp":1723625668130,"operation":"CREATE OR REPLACE TABLE AS SELECT","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutputRows":"6","numOutputBytes":"858"},"engineInfo":"Apache-Spark/3.2.1 Delta-Lake/2.0.0","txnId":"5e49e955-4dca-4948-89a1-32cf38bb834f"}}


In [30]:
# 테이블 드랍의 경우 모든 메타 정보와 데이터 파일은 유지되며, 데이터베이스에 테이블 정보만 삭제됨
spark.sql("DROP TABLE IF EXISTS taxidb.rateCard")

""


In [31]:
!ls {work_dir}/data/rateCard.delta/_delta_log
!cat {work_dir}/data/rateCard.delta/_delta_log/00000000000000000001.json

00000000000000000000.json  00000000000000000001.json
{"add":{"path":"part-00000-879dbb89-ee5a-47d4-ab0d-0e6445663607-c000.snappy.parquet","partitionValues":{},"size":858,"modificationTime":1723625669220,"dataChange":true,"stats":"{\"numRecords\":6,\"minValues\":{\"rateCodeId\":1,\"rateCodeDesc\":\"Group ride\"},\"maxValues\":{\"rateCodeId\":6,\"rateCodeDesc\":\"Standard Rate\"},\"nullCount\":{\"rateCodeId\":0,\"rateCodeDesc\":0}}"}}
{"commitInfo":{"timestamp":1723625668130,"operation":"CREATE OR REPLACE TABLE AS SELECT","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutputRows":"6","numOutputBytes":"858"},"engineInfo":"Apache-Spark/3.2.1 Delta-Lake/2.0.0","txnId":"5e49e955-4dca-4948-89a1-32cf38bb834f"}}


In [32]:
spark.sql("use taxidb")
spark.sql("show tables")

namespace,tableName,isTemporary


In [ ]:
# Generated Columns from spark 3.4.0
spark.sql(f"""
CREATE TABLE taxidb.YellowTaxis (
    RideId INT COMMENT 'This is our primary Key column',
    VendorId INT,
    PickupTime TIMESTAMP,
    PickupYear INT GENERATED ALWAYS AS(YEAR (PickupTime)),
    PickupMonth INT GENERATED ALWAYS AS(MONTH (PickupTime)),
    PickupDay INT GENERATED ALWAYS AS(DAY (PickupTime)),
    DropTime TIMESTAMP,
    CabNumber STRING COMMENT 'Official Yellow Cab Number'
) USING DELTA
LOCATION '{work_dir}/data/YellowTaxis.delta'
COMMENT 'Table to store Yellow Taxi data'
""")

In [ ]:
spark.sql(f"""
INSERT INTO taxidb.YellowTaxis (RideId, VendorId, PickupTime, DropTime, CabNumber)
VALUES (5, 101, '2021-7-1T8:43:28UTC+3', '2021-7-1T8:43:28UTC+3', '51-986')
""")
# The expression you use in GENERATED ALWAYS AS can be any Spark SQL function that always returns the same result
# when given the same argument values, with a few exceptions we will touch on soon
spark.sql(f"""
SELECT PickupTime, PickupYear, PickupMonth, PickupDay FROM taxidb.YellowTaxis
""").show(truncate=False)

In [ ]:
# Non deterministic function
# uuid() 와 같은 generated column 은 non deterministic expression 으로 사용할 수 없다.
# 아래와 같은 expression 의 경우에도 마찬가지
# - User-defined functions
# - Aggregate functions
# - Window functions
# - Functions returning multiple rows

spark.sql(f"""
CREATE OR REPLACE TABLE default.dummy
    ID STRING GENERATED ALWAYS AS (UUID()),
    Name STRING
) USING DELTA
""")